# First-Level Analysis

This lesson is one of two lessons on how to run conventional mass-univariate analyses (AKA GLM analyses, or informally as 'blobology') in Python. The goal of mass-univariate analysis is to identify voxels in the brain that are 'modulated' or differentially 'activated' by a task.

Pretty much all fMRI analyses involve some sort of analysis on the individual subjects (first-level analysis), which is followed by some sort of group-analysis (second-level analysis). The latter seeks to identify group-level effects of whatever was done in the former. 

This lesson focuses on first-level GLM analysis. We will start by doing a step-by-step walkthrough on one subject. The key here is to really take the time to understand both the concepts and the implementation. When I started, I personally found GLM analyses pretty confusing, so don't worry if you get lost! We will cover each step in detail, one chunk of code at a time.  

Afterwards, we'll run first-level analyses on all of the subjects, such that we end up with one model per subject. These models, which we'll save, will be used as input in the next lesson where we will perform our second-level analysis. The implementation we will use is pretty much what you would do in an actual project. 

---
**A note on other fMRI software**:

Typically, people use software such as AFNI, SPM or FSL for mass-univariate analyses. While these packages serve their purpose well, they come with their own set of hurdles:
- Each software has their own learning curves
- Documentation can be sparse, non-existent, or hard to digest 
- Practical tutorials and examples can be hard to come by  
- Their GUI implementations (FSL and SPM) tend to obfuscate analysis and can be very tedious
- SPM requires a MATLAB license, and AFNI and FSL cannot be run directly on Windows

I recommend trying out one of these software packages in your own time just to get a sense of what's out there. They're full fMRI packages that each come with some excellent tools, which can come in handy. I find myself using FSL and AFNI utilities quite a bit in addition to Python.

---

We will implement mass-univariate analyses using a key package, [nistats](https://nistats.github.io/index.html). Nistats is driven by the same group that drives nilearn. Two things we should know about nilearn:
- It's current version (0.0.1b) is in _alpha_. This means that it is very early in it's development. But the key features we need are well-established.  
- It's actually going to be completely merged with nilearn and turn into a `stats` module in nilearn. It's hard to say exactly when this will happen, but this is useful to know if you plan to continue on after the course. 

Nistats is a great package that makes running mass-univariate analyses straightforward. Importantly, it doesn't hide a lot of details and makes it easy to check your steps. I think it does a great job at keep your analyses transparent. 

Let's start by importing all of our functions. You'll notice that there are many more than previous lessons. We're importing:
- numpy and nibabel as usual
- glob, which will allow us to pattern match for files
- matplotlib just to help with some visualization
- pandas, which we'll use to read and adjust some files
- the necessary functions we need from nistats. These all come from different modules
- pickle, which lets us save Python objects to disk

In [ ]:
import glob
import pickle
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nilearn.plotting import view_img
from nistats.design_matrix import make_first_level_design_matrix
from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix

%matplotlib inline

## 1. In-depth single subject walkthrough

We'll go through each step of performing a first-level analysis of the motor localizer in one subject.  

### 1.1. Setting up the data

We'll first get the preprocessed functional runs of one task produce by fmriprep. We can use Python's glob module to extract all files matching a template. The template we'll provide will give us the preprocessed motor localizer runs.  

In [ ]:
glob_template = ('data/derivatives/fmriprep/sub-01/func/'
                '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz')
func_data = glob.glob(glob_template)

# reorder these based on run number 
func_imgs = sorted(list(func_data))
func_imgs

Next, we will also need the run's motion parameters that were extracted during motion correction in preprocessing. fmriprep stores all of the parameters, along with many others, in the `confounds_regressors.tsv` files. Each run has one of these files. 

We will use Pandas to load in the file into Python and select only the motion parameters. 

In [ ]:
glob_template = ('data/derivatives/fmriprep/sub-01/func/'
                '*task-motor*confounds_regressors.tsv')
conf_files = glob.glob(glob_template)

# reorder these based on run number
conf_files = sorted(list(conf_files))

# load only the 6 motion parameters into python
motion_params = []
for conf in conf_files:
    conf_data = pd.read_table(conf, usecols=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z'])
    motion_params.append(conf_data)

# show the first one
motion_params[0]

### 1.2. Defining the model

Before we set up a model, let's load in a brain mask of the MNI template. This mask will restrict analysis to voxels only inside the brain.

In [ ]:
brain_mask = nib.load('MNI_brain_mask.nii.gz')

Next, we will specify our model. Nistats makes it really easy to specify a GLM. The `FirstLevelModel` class let's you define all your parameters in one step, and then you can use it's methods to fit the model and define contrasts. 

The [online documentation](https://nistats.github.io/modules/generated/nistats.first_level_model.FirstLevelModel.html#nistats.first_level_model.FirstLevelModel) for the `FirstLevelModel` gives a description for each parameter. For our analysis, a few things are required:
- The TR, which is 2s
- The HRF model, in which there are several to choose from (see documentation for the options). We will choose Glover's (1999) canonical HRF along with the temporal derivative. Adding the derivative is often recommended to improve model fit. The option we'll use is 'glover + derivative'. 
- We also want to apply some additional processing steps, on top of what was done with fmriprep. We will scale and temporally filter the data (0.01Hz), and spatially smooth the data using a 6mm FWHM kernel. 
- The brain mask we just loaded

Some of the above options are the default parameters, but we'll explicitly specify them.

In [ ]:
model = FirstLevelModel(t_r=2, hrf_model='glover + derivative', high_pass=.01, 
                        signal_scaling=0, smoothing_fwhm=6,
                        mask_img=brain_mask, verbose=1)
model

### 1.3 The model design

The design matrix is what defines our regressors. We are trying to _predict actual brain activity_ using an a model of activity that we would expect to arise from the task. Each condition (AKA trial type) is defined as a single regressor. In our case, we will have 8 task regressors of interest (Arm, Eye, Finger, Grasp, Mouth, Speech, Toes, Touch). The remaining regressors are there to either improve model fit or regress out noise.

In nistats, there are a few different ways to build a design matrix. You can build one directly using the `make_first_level_design_matrix` function from the `design_matrix` module (see [documentation](https://nistats.github.io/modules/generated/nistats.design_matrix.make_first_level_design_matrix.html#nistats.design_matrix.make_first_level_design_matrix)). I'd recommend trying this in your own time because it's nice to build your design matrix and inspect it before running your analysis.

For this lesson, we'll use the much simpler way of specifying a design matrix: we will by pass in events files when fitting the model. The downside to this approach is that we can only inspect our design matrix _after_ the model has finished fitting.

Let's load in the events files that tell us each trial/block's onset, duration, and condition. There is one events file for each run, which we can load in a for-loop using pandas' `read_table` function.

In [ ]:
event_files = glob.glob('data/task-motor*events.tsv')
event_files = sorted(list(event_files))

events = []
for ev in event_files:
    events.append(pd.read_table(ev))

# show the first 10 rows
events[0].head(10)

We see that this is a block design (each block 20s each) with  and that there are 32 blocks (4 each condition). When we pass this DataFrame into our model, it will be transformed to a proper design matrix.

### 1.4 Fitting the model

We fit the model by calling the `.fit()` method. In this method, we pass in the functional image, our events data, and the motion parameters that we saved earlier. 

In [ ]:
model.fit(func_imgs, events=events, confounds=motion_params)

Now that we've run our model, we can inspect the design matrix. _It's really important to check your design matrix to ensure that it is correct_. Small errors, which can easily be spotted by looking at the design matrix in the design, can have large effects on your results.

In [ ]:
design = model.design_matrices_[0]
fig, ax = plt.subplots(figsize=(13, 8))
plot_design_matrix(design, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(design['Arm'])
ax.set(title='Arm', xlabel='Time (s)', ylabel='Signal (a.u.)');

The design matrix accurately reflects what we saw in the events file. 

### 1.5 Inspecting regression coefficients

In [ ]:
beta_map = model.compute_contrast('Arm', output_type='effect_size')
view_img(beta_map, resampling_interpolation='nearest', vmax=3)

## 2. Putting it all together 

We'll copy and paste what we did earlier all into one notebook cell. Everything will be in one for-loop that iterates through each subject. Exactly like before, we'll do the following for each subject:
1. Get the subject's functional images and load them
2. Get the subject's 6 motion parameters
3. Set up a model identical to the one we were using before

The events files are the same across subjects, so we don't need to load those in again. We can just reuse them for each subject. 

In the glob templates for steps 1) and 2), you'll notice that the string starts with an `f` and contains curly braces: `{subject}`. Python will place the `subject` variable of the iteration into the string (e.g., `'01` in the first iteration of the for-loop). 

Note: If you're unfamiliar with for-loops or large chunks of code, don't be alarmed. Take the time to go through each line and identify it's purpose. All we're doing here is applying the same things as earlier, just one at a time for all subjects. 

In [ ]:
subject_numbers = ['01', '02', '03', '04', '05', '06', '07', '08']
columns = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']

glms = []
for subject in subject_numbers:
    
    ## 1. Get the functional images in order
    glob_template = (f'data/derivatives/fmriprep/sub-{subject}/func/'
                     '*task-motor*MNI152NLin2009cAsym*preproc_bold.nii.gz')
    func_imgs = glob.glob(glob_template)
    func_imgs = sorted(list(func_imgs))
    
    # 2. Get the regressors
    glob_template = (f'data/derivatives/fmriprep/sub-{subject}/func/'
                      '*task-motor*confounds_regressors.tsv')
    conf_files = glob.glob(glob_template)
    # put files in order
    conf_files = sorted(list(conf_files))

    # load only the 6 motion parameters
    motion_params = []
    for conf in conf_files:
        conf_data = pd.read_table(conf, usecols=columns)
        motion_params.append(conf_data)
    
    
    # 3. run model with the same parameters
    model = FirstLevelModel(t_r=2, hrf_model='glover + derivative', high_pass=.01, 
                           signal_scaling=0, smoothing_fwhm=6, mask_img=brain_mask)
    # events are the same as we used in our waltkthrough
    model.fit(func_imgs, events=events, confounds=motion_params)
    glms.append(model)
    

### 2.1 Saving the models 

We'll save these models to a file to use them for next week when perform a second-level analysis and discuss multiple comparisons. We can save Python objects to disk using the [pickle package](https://docs.python.org/3/library/pickle.html) that comes as part of the Python standard library. A pickle file (`.p`) is Python's equivalent to MATLAB's `.mat` or R's `.Rdata`. Much like how we can pickle a vegetable for long-term storage, we can pickle Python objects :). 

The file we will make will be ~6GB because it contains everything belonging to our first-level models. Make sure you have the space available. In a typical project it makes sense to combine first and second level analysis into the same script so that you're not saving off unnecessary files.

In [ ]:
with open('first_level_models.p', 'wb') as f:
    pickle.dump(glms, f)

## 3. Excercise: Tinkering around with paramters

Go back to the step-by-step walkthrough and adjust the paremeters. Try different temporal filtering, scaling, spatial smoothing, HRF models, confound regressors, etc. Refer to the documentation to see what parameters are possible. See what happens to the beta map each time -- are there any substantial differences? 